In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.preprocessing import image

from collections import defaultdict

import time
import os
from PIL import Image
#Hash images using the ImageHash library
#https://pypi.python.org/pypi/ImageHash
import imagehash

# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

Using TensorFlow backend.


In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#import os
import sys
module_path = os.path.abspath('src')
if module_path not in sys.path:
    sys.path.append(module_path)

In [67]:
from src.image_manager import ImageManager
from src.scrape_images import ImageScraper

In [7]:
image_df = pd.read_csv('data/image_log_20180204.csv', sep='|', index_col=0)
image_df.head()

,p_hash,filename,folder,time_added,time_verified
0,daf0881497fa12ed,image_sitka_spruce_forest_92.png,picea_sitchensis,2018-02-04 22:17:11.009109,2018-02-04 22:33:01.007864
1,f3b33d68046c2d4d,image_sitka_spruce_forest_86.png,picea_sitchensis,2018-02-04 22:17:11.031102,2018-02-04 22:33:01.027029
2,99a7465a1e99acb4,image_picea_sitchensis_27.png,picea_sitchensis,2018-02-04 22:17:11.049848,2018-02-04 22:33:01.046446
3,b4e9258745b48aeb,image_sitka_spruce_forest_51.png,picea_sitchensis,2018-02-04 22:17:11.060264,2018-02-04 22:33:01.067552
4,f8d0fde0fc906990,image_sitka_spruce_forest_45.png,picea_sitchensis,2018-02-04 22:17:11.068754,2018-02-04 22:33:01.089674


In [8]:
image_df.loc[0,'p_hash']

'daf0881497fa12ed'

In [9]:
type(image_df.loc[0,'p_hash'])

str

In [10]:
im_hash = imagehash.hex_to_hash('daf0881497fa12ed')
im_hash

array([[ True,  True, False,  True,  True, False,  True, False],
       [ True,  True,  True,  True, False, False, False, False],
       [ True, False, False, False,  True, False, False, False],
       [False, False, False,  True, False,  True, False, False],
       [ True, False, False,  True, False,  True,  True,  True],
       [ True,  True,  True,  True,  True, False,  True, False],
       [False, False, False,  True, False, False,  True, False],
       [ True,  True,  True, False,  True,  True, False,  True]], dtype=bool)

In [12]:
## Fails, tries to access .hash of the string
#im_hash == 'daf0881497fa12ed'

In [57]:
type(image_df.loc[0,'time_added'])

str

In [59]:
pd.Timestamp(image_df.loc[0,'time_added'])

Timestamp('2018-02-04 22:17:11.009109')

In [13]:
syncs_df = pd.read_csv('data/image_syncs_20180204.csv', sep='|', index_col=0)
syncs_df

,time_started,time_completed
0,2018-02-04 22:17:10.870517,2018-02-04 22:17:17.920112
1,2018-02-04 22:32:31.024992,2018-02-04 22:33:45.647715


In [17]:
syncs_df.loc[0, 'folders'] = repr(['picea_sitchensis', 'bad_images'])
syncs_df

,time_started,time_completed,folders
0,2018-02-04 22:17:10.870517,2018-02-04 22:17:17.920112,"['picea_sitchensis', 'bad_images']"
1,2018-02-04 22:32:31.024992,2018-02-04 22:33:45.647715,nan


In [18]:
syncs_df.loc[0, 'folders']

"['picea_sitchensis', 'bad_images']"

In [20]:
folders = eval(syncs_df.loc[0, 'folders'])
folders

In [22]:
folders[0]

'picea_sitchensis'

In [23]:
repr(im_hash)

'array([[ True,  True, False,  True,  True, False,  True, False],\n       [ True,  True,  True,  True, False, False, False, False],\n       [ True, False, False, False,  True, False, False, False],\n       [False, False, False,  True, False,  True, False, False],\n       [ True, False, False,  True, False,  True,  True,  True],\n       [ True,  True,  True,  True,  True, False,  True, False],\n       [False, False, False,  True, False, False,  True, False],\n       [ True,  True,  True, False,  True,  True, False,  True]], dtype=bool)'

In [24]:
type(syncs_df.loc[0, 'time_completed'])

str

In [25]:
timstamp0 = pd.Timestamp(syncs_df.loc[0, 'time_completed'])
type(timstamp0)

pandas._libs.tslib.Timestamp

In [26]:
timstamp0

Timestamp('2018-02-04 22:17:17.920112')

In [27]:
syncs_df['time_started'].apply(pd.Timestamp)

0   2018-02-04 22:17:10.870517
1   2018-02-04 22:32:31.024992
Name: time_started, dtype: datetime64[ns]

In [28]:
type(syncs_df.loc[0, 'time_started'])

str

In [40]:
syncs_df.loc[:,['time_started', 'time_completed']] = syncs_df.loc[:,['time_started', 'time_completed']].applymap(pd.Timestamp)

In [41]:
syncs_df

,time_started,time_completed,folders
0,1517782630870517000,1517782637920112000,"['picea_sitchensis', 'bad_images']"
1,1517783551024992000,1517783625647715000,nan


In [42]:
type(syncs_df.loc[0, 'time_started'])

int

In [43]:
type(syncs_df.loc[0, 'folders'])

str

In [53]:
# syncs_df.loc[:,'folders'] = syncs_df.loc[:,'folders'].apply(eval)
# syncs_df

In [51]:
type(syncs_df.loc[0,'folders'])

str

In [54]:
syncs_df

,time_started,time_completed,folders
0,1517782630870517000,1517782637920112000,"['picea_sitchensis', 'bad_images']"
1,1517783551024992000,1517783625647715000,nan


In [56]:
image_df['p_hash'].apply(imagehash.hex_to_hash)[0]

array([[ True,  True, False,  True,  True, False,  True, False],
       [ True,  True,  True,  True, False, False, False, False],
       [ True, False, False, False,  True, False, False, False],
       [False, False, False,  True, False,  True, False, False],
       [ True, False, False,  True, False,  True,  True,  True],
       [ True,  True,  True,  True,  True, False,  True, False],
       [False, False, False,  True, False, False,  True, False],
       [ True,  True,  True, False,  True,  True, False,  True]], dtype=bool)

In [75]:
manager = ImageManager('tree_photos')
manager.load_image_df('data/image_log_20180204.csv')

In [76]:
manager.image_df.head()

,p_hash,filename,folder,time_added,time_verified
0,daf0881497fa12ed,image_sitka_spruce_forest_92.png,picea_sitchensis,1517782631009109000,1517783581007864000
1,f3b33d68046c2d4d,image_sitka_spruce_forest_86.png,picea_sitchensis,1517782631031102000,1517783581027029000
2,99a7465a1e99acb4,image_picea_sitchensis_27.png,picea_sitchensis,1517782631049848000,1517783581046446000
3,b4e9258745b48aeb,image_sitka_spruce_forest_51.png,picea_sitchensis,1517782631060264000,1517783581067552000
4,f8d0fde0fc906990,image_sitka_spruce_forest_45.png,picea_sitchensis,1517782631068754000,1517783581089674000


In [71]:
type(manager.image_df.loc[0,'p_hash'])

imagehash.ImageHash

In [72]:
type(manager.image_df.loc[0,'time_added'])

int

In [73]:
manager.image_df.loc[0,'time_added']

1517782631009109000

In [74]:
pd.Timestamp(manager.image_df.loc[0,'time_added'])

Timestamp('2018-02-04 22:17:11.009109')

In [79]:
fake_hash = imagehash.hex_to_hash('aaaabbbbcccddd')
fake_hash

array([[ True, False,  True, False,  True, False,  True],
       [False,  True, False,  True, False,  True, False],
       [ True, False,  True, False,  True,  True,  True],
       [False,  True,  True,  True, False,  True,  True],
       [ True, False,  True,  True,  True,  True, False],
       [False,  True,  True, False, False,  True,  True],
       [False, False,  True,  True, False,  True,  True],
       [ True, False,  True,  True,  True, False,  True]], dtype=bool)

In [81]:
manager.image_df.loc[manager.image_df['p_hash'] == fake_hash]

,p_hash,filename,folder,time_added,time_verified


In [82]:
manager.image_df.loc[manager.image_df['p_hash'] == fake_hash, 'p_hash'] = fake_hash

In [83]:
manager.image_df.loc[manager.image_df['p_hash'] == fake_hash]

,p_hash,filename,folder,time_added,time_verified
